In [1]:
path="/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-05-28-16-36-15"

In [2]:
import pickle
import os
import re
import pprint 
pp = pprint.PrettyPrinter(indent=4)

In [3]:
import pickle
experimentfile = os.path.join(path,filter(lambda s:s.endswith('exp'),os.listdir(path))[0])

exp=pickle.load(open(experimentfile,'rb'))

OSError: [Errno 2] No such file or directory: '/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-05-28-16-36-15'

In [ ]:
exp.print_stats(exp)

In [ ]:
import pandas as pd
import plotly.plotly as py
import statsmodels as sm

from sklearn.linear_model import LinearRegression
import scipy, scipy.stats

import cufflinks as cf # Awesome Pandas/Plot.ly integration module https://github.com/santosjorge/cufflinks

py.sign_in('bolster', 'q54pd5bbmd')

from bounos import Analyses, Metrics

_metrics = [Metrics.DeviationOfHeading,
            Metrics.PernodeSpeed,
            Metrics.PernodeInternodeDistanceAvg]
run = 0
deviance_s, windowed_s = Analyses.Behaviour.combined_detection_rank(exp.scenarios[ 'Behaviour(Shadow)'].datarun[run], _metrics)
deviance_w, windowed_w = Analyses.Behaviour.combined_detection_rank(exp.scenarios[ 'Behaviour(Waypoint)'].datarun[run], _metrics)
deviance_sc, windowed_sc = Analyses.Behaviour.combined_detection_rank(exp.scenarios[ 'Behaviour(SlowCoach)'].datarun[run], _metrics)

In [ ]:
import bounos
from pprint import pprint
reload(bounos)
pprint(Analyses.Behaviour.behaviour_identification(
        deviance_w, windowed_w, _metrics))
pprint(Analyses.Behaviour.behaviour_identification(
        deviance_s, windowed_s, _metrics))
pprint(Analyses.Behaviour.behaviour_identification(
        deviance_sc, windowed_sc, _metrics))

In [ ]:
df_w = pd.DataFrame.from_records(windowed_w)
df_w.iplot(filename='Windowed Trust Dev Waypoint',world_readable=True, theme="pearl")
#dft = pd.concat([pd.DataFrame.from_records(windowed[i]) for i in range(len(_metrics))], keys=[m.__name__ for m in _metrics], names=['Metric','Time'])

In [ ]:
dfnorm(df_w).sum(axis=0).std()

In [ ]:
df_s = pd.DataFrame.from_records(windowed_s)
df_s.iplot(filename='Windowed Trust Dev Shadow',world_readable=True, theme="pearl",)

In [ ]:
dfnorm(df_s).sum(axis=0).std()

In [ ]:
df_sc = pd.DataFrame.from_records(windowed_sc)
df_sc.iplot(filename='Windowed Trust Dev SlowCoach',world_readable=True, theme="pearl",)

In [ ]:
df2norm(df_sc).sum(axis=0).std()

In [ ]:
def df2norm(df): return (df - df.mean().mean()) / (df.max().max() - df.min().min())
dfn_w, dfn_s, dfn_sc = map(df2norm, [df_w, df_s, df_sc])
dfn_s.plot()

In [ ]:
df_w.max().max()

In [ ]:
import pickle
with open(os.path.join(path, "panel.pkl"), 'rb') as f:
    panel = pickle.load(f)

for name,frame in panel.iteritems():
    anonframe=frame.drop('suspect_name', 1)
    filterkey='suspect_confidence'
    if "Waypoint" in name:
        bestrun = anonframe.idxmin(axis=0)[filterkey]
    else:
        bestrun = anonframe.idxmax(axis=0)[filterkey]
    print name, bestrun

In [ ]:
import pickle, os
import bounos as b
args = b.custom_parser().parse_known_args()[0]
with open(os.path.join(path,'bestruns.pkl'), 'rb') as f:
    runs = pickle.load(f)
args

In [ ]:
reload(b)
args.annotate_achievements=1
args.strip_title=True

b.custom_fusion_run(data=runs, args=args, title='fuse')


In [ ]:
metric_deviance, windowed_accumulator = b.Analyses.Combined_Detection_Rank(runs.values()[0], b._metrics)


In [ ]:
for metric in metric_deviance:
    figure()
    plot(metric)

In [ ]:
raw_metrics = map(lambda f:f(),b._metrics)
map(lambda f:f.update(runs.values()[0]),raw_metrics)
pns = raw_metrics[1]
rvals = range(150,180)
rvals=range(1000)

In [ ]:
dev_results =b.Analyses.Deviation(datapackage=runs.values()[0],metric=pns)
metrics, stddev, deviance = dev_results.values()
delta = [pns.data[t]-pns.highlight_data[t] for t in rvals]
eps=np.finfo(np.float64).eps
std=np.std(delta, axis=1).clip(min=eps)
stddev = [ np.abs(delta[t]/std[t]) for t in range(len(rvals))]
figure()
plot([pns.data[t] for t in rvals])
plot([pns.highlight_data[t] for t in rvals],'--')

figure()
plot(stddev)
figure()
plot(map(abs,delta))
plot(std,'--')


In [ ]:
std[7]

In [ ]:
eps

In [ ]:
plot(np.average(pns.data, axis=1)-pns.highlight_data)

In [ ]:
plot(pns.highlight_data)